<a href="https://colab.research.google.com/github/sas775/ProgrammingAssignment2/blob/master/Rent_Cost_Bangladesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mp
import matplotlib.pyplot as plt
import sklearn

In [48]:
file_path = '/content/Property Rent Data.xlsx'  # Update the path as needed
df = pd.read_excel(file_path)


In [49]:
df.head()

,Address,Area in sq foot,Bath,Beds,Last Update Date,Sum of Rent Price,Type
0,Aftab Nagar,1000,2,3,2022-08-24,17,Apartment
1,Aftab Nagar,1100,2,2,2023-02-01,54,Apartment
2,Aftab Nagar,1100,2,2,2023-02-14,36,Apartment
3,Aftab Nagar,1100,2,2,2023-02-22,18,Apartment
4,Aftab Nagar,1100,3,3,2022-09-30,20,Apartment


In [50]:
!pip install pandas openpyxl fuzzywuzzy

from fuzzywuzzy import fuzz
from collections import defaultdict



In [38]:

# Define a function to compute similarity percentage
def compute_similarity(address1, address2):
    """Compute similarity percentage between two addresses."""
    return fuzz.ratio(address1.lower(), address2.lower())

# Define a function to find clusters based on similarity
def find_clusters(addresses, threshold=80):
    """Find clusters of addresses based on a similarity threshold."""
    clusters = defaultdict(list)
    cluster_id = 1

    address_to_cluster = {}

    for i, address in enumerate(addresses):
        if address in address_to_cluster:
            continue

        # Start a new cluster
        cluster = []
        for j, other_address in enumerate(addresses):
            if i != j and compute_similarity(address, other_address) >= threshold:
                cluster.append(other_address)
                address_to_cluster[other_address] = cluster_id

        if cluster:
            cluster.append(address)
            for addr in cluster:
                address_to_cluster[addr] = cluster_id
            cluster_id += 1

    # Assign cluster IDs to unique addresses that didn't get a cluster ID
    for i, address in enumerate(addresses):
        if address not in address_to_cluster:
            address_to_cluster[address] = cluster_id
            cluster_id += 1

    return address_to_cluster

# Extract unique addresses for clustering
addresses = df['Address'].tolist()

# Initialize the address_to_cluster dictionary
address_to_cluster = {}

# Find clusters
address_to_cluster = find_clusters(addresses, threshold=80)

# Map addresses to their cluster IDs
df['Cluster'] = df['Address'].map(address_to_cluster)

# Insert the new column right after the first column
df.insert(1, 'Cluster', df.pop('Cluster'))

# Save the updated DataFrame to a new Excel file
output_file_path = '/content/Rent_Property_Data_Clustered.xlsx'
df.to_excel(output_file_path, index=False)

# Provide the link to download the file
output_file_path


'/content/Rent_Property_Data_Clustered.xlsx'

In [246]:
file_path = '/content/Rent_Property_Data_Clustered.xlsx'  # Update the path as needed
df = pd.read_excel(file_path)
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[6], axis=1)
df = df[df.iloc[:, 5] <= 250]
from datetime import datetime
# Convert the "Last Update Date" column to datetime
df['Last Update Date'] = pd.to_datetime(df['Last Update Date'])
# Get the current date and time
now = datetime.now()

# Calculate the difference from the current date
df['Difference from Now (days)'] = (now - df['Last Update Date']).dt.total_seconds() / (24 * 3600)
df.head()


,Cluster,Area in sq foot,Bath,Beds,Last Update Date,Sum of Rent Price,Difference from Now (days)
0,1,1000,2,3,2022-08-24,17,692.778104
1,1,1100,2,2,2023-02-01,54,531.778104
2,1,1100,2,2,2023-02-14,36,518.778104
3,1,1100,2,2,2023-02-22,18,510.778104
4,1,1100,3,3,2022-09-30,20,655.778104


In [247]:
y = df.iloc[:, 5]
y.head()

0    17
1    54
2    36
3    18
4    20
Name: Sum of Rent Price, dtype: int64

In [248]:
x = df.iloc[:, [0,1,2,3,6]]
print(x)

      Cluster  Area in sq foot  Bath  Beds  Difference from Now (days)
0           1             1000     2     3                  692.778104
1           1             1100     2     2                  531.778104
2           1             1100     2     2                  518.778104
3           1             1100     2     2                  510.778104
4           1             1100     3     3                  655.778104
...       ...              ...   ...   ...                         ...
6080      239             2100     3     3                  556.778104
6081      239             2115     3     3                  992.778104
6082      239             2200     4     4                  611.778104
6083      239             2200     5     4                  521.778104
6084      239             2500     4     4                  570.778104

[5992 rows x 5 columns]


In [249]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb


In [240]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)


In [251]:
model = KNeighborsRegressor(n_neighbors=15)
#model = RandomForestRegressor(n_estimators=100, random_state=42)
#model = XGBRegressor(random_state=42)
#model = GradientBoostingRegressor(random_state=42)
#model = Lasso(random_state=42)
#model = lgb.LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')



Mean Squared Error: 1447.673855232826
R^2 Score: 0.23759451344861504
Mean Absolute Error: 22.83319502074689
